In [1]:
#import libraries 
import matplotlib.pyplot as plt
import numpy as np

import skimage.io
from skimage.color import rgb2hsv
from skimage.exposure import equalize_adapthist

import cv2

import os

from rembg.bg import remove
import io
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# print(os.listdir("rice_leaf_diseases"))


In [4]:
#path folder penyakit
ori_blb_path = "rice_leaf_diseases/train/bacterial_leaf_blight/"
ori_bs_path = "rice_leaf_diseases/train/brown_spot/"
ori_ls_path = "rice_leaf_diseases/train/leaf_smut/"
ori_healthy_path = "rice_leaf_diseases/train/healthy/"


In [ ]:
import colorcorrect.algorithm as cca
from colorcorrect.util import from_pil, to_pil


def colorBalance(image, name, path):
  """
  Implement cca.max_white color correction algorithm 
    using colorcorrect library
  Args:
        image: image object that will be color balanced
        name: filename of the saved color balanced image
        path: color balanced image destination folder
  Returns: 
  """
  try:
    to_pil(cca.max_white(from_pil(image))).save(path + name)

  except Exception as e:
    print(f"Error : {e}")


In [ ]:
# path setelah color balanced
cb_blb_path = "rld_img_cb/blb/"
cb_bs_path = "rld_img_cb/bs/"
cb_ls_path = "rld_img_cb/ls/"
cb_healthy_path = "rld_img_cb/healthy/"

# image color balancing
# blb_os_path = os.listdir(ori_blb_path)
# for img_name in blb_os_path[:]:
#     read_img = Image.open(ori_blb_path + img_name)
#     colorBalance(read_img, img_name, cb_blb_path)

# bs_os_path = os.listdir(ori_bs_path)
# for img_name in bs_os_path[:]:
#     read_img = Image.open(ori_bs_path + img_name)
#     colorBalance(read_img, img_name, cb_bs_path)

# ls_os_path = os.listdir(ori_ls_path)
# for img_name in ls_os_path[:]:
#     read_img = Image.open(ori_ls_path + img_name)
#     colorBalance(read_img, img_name, cb_ls_path)

# healthy_os_path = os.listdir(ori_healthy_path)
# for img_name in healthy_os_path[:]:
#     read_img = Image.open(ori_healthy_path + img_name)
#     colorBalance(read_img, img_name, cb_healthy_path)

In [2]:
from skimage.morphology import binary_closing, binary_opening, erosion
import numpy as np

In [3]:
selem = np.zeros((25, 25))

ci,cj=12, 12
cr=13

# Create index arrays to z
I,J=np.meshgrid(np.arange(selem.shape[0]),np.arange(selem.shape[1]))

# calculate distance of all points to centre
dist=np.sqrt((I-ci)**2+(J-cj)**2)

# Assign value of 1 to those points where dist<cr:
selem[np.where(dist<=cr)]=1

In [4]:
%matplotlib inline
import numpy as np
from scipy import ndimage
from PIL import Image
import os

In [11]:
ori_blb_path = "rice_leaf_diseases/train/bacterial_leaf_blight/"
ori_bs_path = "rice_leaf_diseases/train/brown_spot/"
ori_ls_path = "rice_leaf_diseases/train/leaf_smut/"

r_blb_path = "bgr_rmv_erosion/blb/train/"
r_bs_path = "bgr_rmv_erosion/bs/train/"
r_ls_path = "bgr_rmv_erosion/ls/train/"


e_blb_path = "erosion_mask/blb/"
e_bs_path = "erosion_mask/bs/"
e_ls_path = "erosion_mask/ls/"



Background Removal Erosion is based from this website "https://github.com/ejmmedina/rice-disease-classification/blob/master/riceimg-clf-ml2.ipynb"

In [17]:
blb_os_path = os.listdir(ori_blb_path)
for img_name in blb_os_path[:]:
    bg_frac = 0
    thres = 220
    img = Image.open(ori_blb_path + img_name)
    im_arr = np.array(img)

    R = im_arr[:, :, 0]
    G = im_arr[:, :, 1]
    B = im_arr[:, :, 2]
    while bg_frac < 0.2: 
        bg_mask = ((R>thres) | (B>thres))# & (G < 100)
        bg_frac = bg_mask.sum()/len(bg_mask.flatten())
        thres -= 5
    # we use opening first since our mask is reversed (the foreground and background are reversed here)
    bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

     #Get biggest blob
    label, num_label = ndimage.label(~bg_mask)
    size = np.bincount(label.ravel())
    biggest_label = size[1:].argmax() + 1
    bg_mask = label == biggest_label
    
    im_arr[~bg_mask, 0] = 0
    im_arr[~bg_mask, 1] = 0
    im_arr[~bg_mask, 2] = 0
    
    img = Image.fromarray(im_arr)
    mask_img = Image.fromarray(bg_mask)
    # img.save(r_blb_path + img_name)
    mask_img.save(e_blb_path + img_name)
   

In [15]:
bs_os_path = os.listdir(ori_bs_path)
for img_name in bs_os_path[:]:
    bg_frac = 0
    thres = 220
    img = Image.open(ori_bs_path + img_name)
    im_arr = np.array(img)
#     ax[idx, 0].imshow(im_arr)
    R = im_arr[:, :, 0]
    G = im_arr[:, :, 1]
    B = im_arr[:, :, 2]
    while bg_frac < 0.2: 
        bg_mask = ((R>thres) | (B>thres))# & (G < 100)
        bg_frac = bg_mask.sum()/len(bg_mask.flatten())
        thres -= 5
    # we use opening first since our mask is reversed (the foreground and background are reversed here)
    bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

     #Get biggest blob
    label, num_label = ndimage.label(~bg_mask)
    size = np.bincount(label.ravel())
    biggest_label = size[1:].argmax() + 1
    bg_mask = label == biggest_label
    
    im_arr[~bg_mask, 0] = 0
    im_arr[~bg_mask, 1] = 0
    im_arr[~bg_mask, 2] = 0
    
    img = Image.fromarray(im_arr)
    mask_img = Image.fromarray(bg_mask)
    # img.save(r_blb_path + img_name)
    mask_img.save(e_bs_path + img_name)
   

In [16]:
ls_os_path = os.listdir(ori_ls_path)
for img_name in ls_os_path[:]:
    bg_frac = 0
    thres = 220
    img = Image.open(ori_ls_path + img_name)
    im_arr = np.array(img)

    R = im_arr[:, :, 0]
    G = im_arr[:, :, 1]
    B = im_arr[:, :, 2]
    while bg_frac < 0.2: 
        bg_mask = ((R>thres) | (B>thres))# & (G < 100)
        bg_frac = bg_mask.sum()/len(bg_mask.flatten())
        thres -= 5
    # we use opening first since our mask is reversed (the foreground and background are reversed here)
    bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

     #Get biggest blob
    label, num_label = ndimage.label(~bg_mask)
    size = np.bincount(label.ravel())
    biggest_label = size[1:].argmax() + 1
    bg_mask = label == biggest_label
    

    #background black
    im_arr[~bg_mask, 0] = 0
    im_arr[~bg_mask, 1] = 0
    im_arr[~bg_mask, 2] = 0
    
    img = Image.fromarray(im_arr)
    mask_img = Image.fromarray(bg_mask)
    # img.save(r_blb_path + img_name)
    mask_img.save(e_ls_path + img_name)
   

In [ ]:
healthy_os_path = os.listdir(ori_healthy_path)
for img_name in healthy_os_path[:]:
    bg_frac = 0
    thres = 220
    img = Image.open(ori_healthy_path + img_name)
    im_arr = np.array(img)
#     ax[idx, 0].imshow(im_arr)
    R = im_arr[:, :, 0]
    G = im_arr[:, :, 1]
    B = im_arr[:, :, 2]
    while bg_frac < 0.2: 
        bg_mask = ((R>thres) | (B>thres))# & (G < 100)
        bg_frac = bg_mask.sum()/len(bg_mask.flatten())
        thres -= 5
    # we use opening first since our mask is reversed (the foreground and background are reversed here)
    bg_mask = binary_closing(erosion(binary_opening(bg_mask, selem), np.ones((3, 3))), np.ones((5,5)))

     #Get biggest blob
    label, num_label = ndimage.label(~bg_mask)
    size = np.bincount(label.ravel())
    biggest_label = size[1:].argmax() + 1
    bg_mask = label == biggest_label
    
    im_arr[~bg_mask, 0] = 0
    im_arr[~bg_mask, 1] = 0
    im_arr[~bg_mask, 2] = 0
    
    img = Image.fromarray(im_arr)
    img.save(r_healthy_path + img_name)
   

In [16]:
# path mask
m_blb_path = "rld_mask/blb/"
m_bs_path = "rld_mask/bs/"
m_ls_path = "rld_mask/ls/"
m_healthy_path = "rld_mask/healthy/"


def create_hue_mask(hsv_img, lower_hue_val, upper_hue_val):
    """
    Create mask of images from path_r and save them to 
        path_m using hue segmentation
    Args:
        hsv_img: image in HSV 
        lower_hue_val: lowest hue threshold value
        upper_hue_val: highest hue threshold value
    Returns: 
    """
    try:     
        # Hue segment
        img_hue = hsv_img[:,:,0]
        l_mask = img_hue > lower_hue_val
        u_mask = img_hue < upper_hue_val
        hue_mask = u_mask * l_mask
        return hue_mask
    except Exception as e:
        print(f"Error : {e}")
        

In [17]:

blb_os_path = os.listdir(r_blb_path)
for img_name in blb_os_path[:]:
    try: 
        read_img = cv2.imread(r_blb_path + img_name) #read image from path
        read_rgb_img = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
        read_hsv_img = rgb2hsv(read_rgb_img)

        mask_img = create_hue_mask(read_hsv_img, 0.05, 0.15)
        plt.imsave(m_blb_path + img_name, mask_img, cmap="gray") 
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

bs_os_path = os.listdir(r_bs_path)
for img_name in bs_os_path[:]:
    try: 
        read_img = cv2.imread(r_bs_path + img_name) #read image from path
        read_rgb_img = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
        read_hsv_img = rgb2hsv(read_rgb_img)

        mask_img = create_hue_mask(read_hsv_img, 0.0, 0.1)
        plt.imsave(m_bs_path + img_name, mask_img, cmap="gray") 
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

ls_os_path = os.listdir(r_ls_path)
for img_name in ls_os_path[:]:
    try: 
        read_img = cv2.imread(r_ls_path + img_name) #read image from path
        read_rgb_img = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
        read_hsv_img = rgb2hsv(read_rgb_img)

        mask_img = create_hue_mask(read_hsv_img, 0.0, 0.1)
        plt.imsave(m_ls_path + img_name, mask_img, cmap="gray") 
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

healthy_os_path = os.listdir(r_healthy_path)
for img_name in healthy_os_path[:]:
    try:
        read_img = cv2.imread(r_healthy_path + img_name) #read image from path
        
        read_rgb_img = cv2.cvtColor(read_img, cv2.COLOR_BGR2RGB)
        read_hsv_img = rgb2hsv(read_rgb_img)

        mask_img = create_hue_mask(read_hsv_img, 0.0, 1.0)
        plt.imsave(m_healthy_path + img_name, mask_img, cmap="gray") 
    except Exception as e:
            print(f"Error : {e} at image {img_name}")





In [18]:
# path segment img
seg_blb_path = "segment_img/blb/"
seg_bs_path = "segment_img/bs/"
seg_ls_path = "segment_img/ls/"
seg_healthy_path = "segment_img/healthy/"


def segment_disease(ori_img, mask_img):
    """
    Segment disease from the paddy leaf
    Args:
        ori_img: original image
        mask_img: mask image 
    Returns: 
        res: segment disease image array
    """

    try: 
        res = cv2.bitwise_and(ori_img, mask_img)
        return res
    except Exception as e:
        print(f"Error : {e}")

In [19]:
r_os_path = os.listdir(r_blb_path)
for img_name in r_os_path[:]:
    try: 
        ori_img = cv2.imread(r_blb_path + img_name) #read ori image from path
        mask_img = cv2.imread(m_blb_path + img_name) #read image from path
        result = segment_disease(ori_img, mask_img)
        cv2.imwrite(seg_blb_path + img_name, result)
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

r_os_path = os.listdir(r_bs_path)
for img_name in r_os_path[:]:
    try: 
        ori_img = cv2.imread(r_bs_path + img_name) #read ori image from path
        mask_img = cv2.imread(m_bs_path + img_name) #read image from path
        result = segment_disease(ori_img, mask_img)
        cv2.imwrite(seg_bs_path + img_name, result)
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

r_os_path = os.listdir(r_ls_path)
for img_name in r_os_path[:]:
    try: 
        ori_img = cv2.imread(ori_ls_path + img_name) #read ori image from path
        mask_img = cv2.imread(m_ls_path + img_name) #read image from path
        result = segment_disease(ori_img, mask_img)
        cv2.imwrite(seg_ls_path + img_name, result)
    except Exception as e:
        print(f"Error : {e} at image {img_name}")

r_os_path = os.listdir(r_healthy_path)
for img_name in r_os_path[:]:
    try: 
        ori_img = cv2.imread(ori_healthy_path + img_name) #read ori image from path
        mask_img = cv2.imread(m_healthy_path + img_name) #read image from path
        result = segment_disease(ori_img, mask_img)
        cv2.imwrite(seg_healthy_path + img_name, result)
    except Exception as e:
            print(f"Error : {e} at image {img_name}")


Image Classifier 5Conv2Dense was based from this site -> https://github.com/ejmmedina/rice-disease-classification/blob/master/riceimg-clf-ml2.ipynb